# Домашнее задание №2. Часть 2.

Собственно, тут мы используем полученные файлы для создания аннотированного генома бактерии в формате GenBank.

## Скачиваем полученные файлы на предыдущем шаге:

In [ ]:
!gdown 1QRXPodBp6RV89Se0ovL8BdtvzgOtwWiq

Downloading...
From: https://drive.google.com/uc?id=1QRXPodBp6RV89Se0ovL8BdtvzgOtwWiq
To: /content/Poil_gapClosed.fa
100% 3.98M/3.98M [00:00<00:00, 181MB/s]


In [ ]:
!gdown 1lWR3HzJuPRPlm1Rsc9J1-1D-LOK-35Hg

Downloading...
From: https://drive.google.com/uc?id=1lWR3HzJuPRPlm1Rsc9J1-1D-LOK-35Hg
To: /content/gms2.lst
100% 221k/221k [00:00<00:00, 133MB/s]


In [ ]:
!gdown 1ZvR9SZF-YJmGUrHJP6-3HV6SlqfiF9xS

Downloading...
From: https://drive.google.com/uc?id=1ZvR9SZF-YJmGUrHJP6-3HV6SlqfiF9xS
To: /content/proteins.fasta
100% 1.40M/1.40M [00:00<00:00, 46.5MB/s]


In [ ]:
!gdown 1CmA4niPhy21tpZLbVembiJ94rChVRaU9

Downloading...
From: https://drive.google.com/uc?id=1CmA4niPhy21tpZLbVembiJ94rChVRaU9
To: /content/scaffolds.hits_from_MIL_1.txt
100% 871k/871k [00:00<00:00, 102MB/s]


In [ ]:
!gdown 1i1lJmzzN_6m1XQ9XrCln72o5ke_gtECF

Downloading...
From: https://drive.google.com/uc?id=1i1lJmzzN_6m1XQ9XrCln72o5ke_gtECF
To: /content/scaffolds.hits_from_SwissProt.txt
100% 66.4k/66.4k [00:00<00:00, 110MB/s]


## Теперь необходимо скачать геном близкородственной бактерии - T.oleivorans

### Установим модуль NCBI:

In [ ]:
!wget ftp://ftp.ncbi.nlm.nih.gov/entrez/entrezdirect/install-edirect.sh

--2023-10-21 20:24:17--  ftp://ftp.ncbi.nlm.nih.gov/entrez/entrezdirect/install-edirect.sh
           => ‘install-edirect.sh’
Resolving ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)... 165.112.9.229, 130.14.250.13, 2607:f220:41e:250::13, ...
Connecting to ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)|165.112.9.229|:21... connected.
Logging in as anonymous ... Logged in!
==> SYST ... done.    ==> PWD ... done.
==> TYPE I ... done.  ==> CWD (1) /entrez/entrezdirect ... done.
==> SIZE install-edirect.sh ... 4165
==> PASV ... done.    ==> RETR install-edirect.sh ... done.
Length: 4165 (4.1K) (unauthoritative)

install-edirect.sh  100%[===================>]   4.07K  --.-KB/s    in 0s      

2023-10-21 20:24:18 (601 MB/s) - ‘install-edirect.sh’ saved [4165]



In [ ]:
!chmod 755 install-edirect.sh

In [ ]:
!./install-edirect.sh


Entrez Direct has been successfully downloaded and installed.

In order to complete the configuration process, please execute the following:

  echo "export PATH=/root/edirect:\${PATH}" >> ${HOME}/.bashrc

or manually edit the PATH variable assignment in your .bashrc file.

Would you like to do that automatically now? [y/N]
^C


In [ ]:
!echo "export PATH=/root/edirect:\${PATH}" >> ${HOME}/.bashrc

In [ ]:
!./install-edirect.sh


Entrez Direct has been successfully downloaded and installed.


To activate EDirect for this terminal session, please execute the following:

export PATH=${HOME}/edirect:${PATH}



In [ ]:
!export PATH=${HOME}/edirect:${PATH}

### Скачиваем геном:

In [ ]:
!$HOME/edirect/efetch -db nuccore -id HF680312 -format gb  >  T_oleivorans_MIL_1.gbk

## Переходим к анотации генома!

### Установим biopython

In [ ]:
!pip install biopython

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 31.0 MB/s eta 0:00:00


### Аннотация:

In [ ]:
from Bio import SeqIO
from Bio.SeqFeature import SeqFeature, FeatureLocation
from datetime import datetime as dt

In [ ]:
scaffolds = dict()
for record in SeqIO.parse("Poil_gapClosed.fa", "fasta"):
  record.annotations['molecule_type'] = 'DNA'
  record.annotations['date'] = dt.now().strftime("%d-%b-%Y").upper()
  record.annotations['data_file_division'] = 'BCT'

  scaffolds[record.id] = record

In [ ]:
genes = dict()
for gene in SeqIO.parse("proteins.fasta", "fasta"):
  desc = gene.description.split(' ')
  scaffold = desc[1]
  start, end = int(desc[2]), int(desc[3])
  strand = 1 if desc[4] == '+' else -1

  feat = SeqFeature(FeatureLocation(start, end, strand=strand), type="CDS")
  feat.qualifiers['locus_tag'] = [desc[0]]
  feat.qualifiers['translation'] = [gene.seq]
  scaffolds[scaffold].features.append(feat)

  genes[desc[0]] = feat

## Теперь добавляем функции белков (из MIL-1)

In [22]:
from Bio import SeqIO

genbank_record = SeqIO.read("T_oleivorans_MIL_1.gbk", "genbank")

mil1genes = {feat.qualifiers['protein_id'][0]: feat.qualifiers['product'][0] for feat in genbank_record.features if 'protein_id' in feat.qualifiers and 'product' in feat.qualifiers}


In [23]:
import pandas as pd

In [25]:
names = ['qseqid', 'sseqid', 'pident', 'length', 'mismatch', 'gapopen', 'qstart', 'qend', 'sstart', 'send', 'evalue', 'bitscore']
mil1hits = pd.read_csv('scaffolds.hits_from_MIL_1.txt', sep='\t', header=None, names=names)
mil1hits.head()

,qseqid,sseqid,pident,length,mismatch,gapopen,qstart,qend,sstart,send,evalue,bitscore
0,1,lcl|HF680312.1_prot_CCU71653.1_1204,98.990,99,1,0,1,99,1,99,8.080000e-68,196.0
1,1,lcl|HF680312.1_prot_CCU72283.1_1834,97.980,99,2,0,1,99,1,99,2.290000e-67,194.0
2,1,lcl|HF680312.1_prot_CCU71933.1_1484,97.980,99,2,0,1,99,1,99,4.150000e-66,191.0
3,1,lcl|HF680312.1_prot_CCU71592.1_1143,97.980,99,2,0,1,99,1,99,4.150000e-66,191.0
4,1,lcl|HF680312.1_prot_CCU71865.1_1416,96.774,62,2,0,1,62,1,62,2.150000e-42,130.0


In [26]:
hits = mil1hits[mil1hits['sseqid'].str.contains("CCU")].sort_values('bitscore', ascending=False).drop_duplicates('qseqid')
hits.head()

,qseqid,sseqid,pident,length,mismatch,gapopen,qstart,qend,sstart,send,evalue,bitscore
2832,1024,lcl|HF680312.1_prot_CCU73896.1_3447,98.979,2253,23,0,1,2253,1,2253,0.0,4296.0
4239,1486,lcl|HF680312.1_prot_CCU70724.1_275,97.870,2113,43,1,7,2119,1,2111,0.0,4158.0
9139,3216,lcl|HF680312.1_prot_CCU71900.1_1451,99.878,1640,2,0,1,1640,1,1640,0.0,3406.0
2609,947,lcl|HF680312.1_prot_CCU73861.1_3412,99.214,1654,13,0,1,1654,1,1654,0.0,3372.0
9918,3493,lcl|HF680312.1_prot_CCU71621.1_1172,99.690,1611,5,0,1,1611,1,1611,0.0,3338.0


In [27]:
for i, hit in hits.iterrows():
  gene = genes[str(hit['qseqid'])]
  match = hit['sseqid'].split('_')[2]
  gene.qualifiers['product'] = [mil1genes[match]]

## Теперь добавляем функции белков (из SwissProt)

### Скачиваем юнипротовскую базу

In [28]:
!wget -nc https://ftp.uniprot.org/pub/databases/uniprot/current_release/knowledgebase/complete/uniprot_sprot.dat.gz
!gzip -d uniprot_sprot.dat.gz

--2023-10-21 21:15:33--  https://ftp.uniprot.org/pub/databases/uniprot/current_release/knowledgebase/complete/uniprot_sprot.dat.gz
Resolving ftp.uniprot.org (ftp.uniprot.org)... 128.175.240.195
Connecting to ftp.uniprot.org (ftp.uniprot.org)|128.175.240.195|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 659315262 (629M) [application/x-gzip]
Saving to: ‘uniprot_sprot.dat.gz’

uniprot_sprot.dat.g 100%[===================>] 628.77M  1.14MB/s    in 9m 6s   

2023-10-21 21:24:39 (1.15 MB/s) - ‘uniprot_sprot.dat.gz’ saved [659315262/659315262]



### Отбираем и сохраняем строки

In [29]:
!grep '^ID\|^DE   RecName: Full=' uniprot_sprot.dat > SwissProt_names.txt

### А теперь немного парсинга

In [30]:
previd = None
swissgenes = dict()
for line in open('SwissProt_names.txt'):
  if line.startswith('ID'):
    previd = line.split()[1]
  if line.startswith('DE'):
    swissgenes[previd] = line.split('=')[1][:-2]

In [31]:
swisshits_raw = pd.read_csv('scaffolds.hits_from_SwissProt.txt', sep='\t', header=None, names=names)
swisshits = swisshits_raw.sort_values('bitscore', ascending=False).drop_duplicates('qseqid')

for i, hit in swisshits.iterrows():
  gene = genes[str(hit['qseqid'])]
  match = hit['sseqid'].split('|')[-1]
  gene.qualifiers['product'] = [swissgenes[match]]

In [32]:
SeqIO.write(scaffolds.values(), "GENOME.gbk", "genbank")

68